## LSTM example

Попробуем построить простейшую LSTM модель на PyTorch. Обучающая выборка будет простой: на вход будут числа X, на выходе ожидаем числа Y

https://github.com/pytorch/examples/blob/master/word_language_model/model.py

https://habr.com/ru/company/wunderfund/blog/331310/

https://pytorch.org/docs/stable/nn.html

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [9]:
# example из официальной документации
# https://pytorch.org/docs/stable/nn.html

rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
output, (hn, cn) = rnn(input)
# нам не нужны h0, c0, hn, cn, скорее всего...

20

In [3]:
# игрушечный датасет
# первая строка -- ожидаемый выходной сигнал, вторая -- входные данные :)
dataset_train = torch.tensor([
    [[1,3,1,4,2,3,3,2], [1,0,0,1,8,4,7,2], [2,4,2,3,2,5,6,3]],
    [[2,2,1,1,4,3,4,2], [2,0,4,2,2,4,2,8], [3,5,3,4,6,1,1,2]],
], dtype=torch.float)


dataset_test= torch.tensor([
    [[1,2,3,1,4,2,3,1], [0,0,2,2,4,1,3,2]],
    [[1,6,2,2,2,2,5,1], [1,4,1,2,5,5,6,5]],
], dtype=torch.float)
# датасет ОПРЕДЕЛЁННО должен быть в формате torch.float

trainloader = torch.utils.data.DataLoader(dataset_train, batch_size=2,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(dataset_test, batch_size=2,
                                         shuffle=False, num_workers=2)

In [4]:
dataset

NameError: name 'dataset' is not defined

In [5]:
# примерно такой shape мы хотим видеть в LSTM:
dataset[:,0,:].transpose(0,1).unsqueeze(2).size()

NameError: name 'dataset' is not defined

In [6]:
# попробуем определить модель LSTM как конечный автомат
class FiniteAutomata(nn.Module):
    def __init__(self):
        super(FiniteAutomata, self).__init__()
        # one input neuron, one output neuron, one layer in LSTM block
        self.input_size = 1
        self.hidden_size = 1
        self.layer_count = 1
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.layer_count)
    
    def forward(self, input):
        # пусть в x  у нас приходит вектор размерности (8, 2, 1)
        # то есть 8 отсчётов, два примера, одно значение в каждом
        output, _ = self.lstm(input)
        return output

In [7]:
fa = FiniteAutomata()

In [8]:
fa.forward(dataset[:,0,:].transpose(0,1).unsqueeze(2))

NameError: name 'dataset' is not defined

Как мы видим, значения генерируются в диапазоне [-1, 1] -- это потому, что активационной функцией LSTM является tanh. Не уверен, можно ли это поменять, но на входе и выходе сети можно влепить дополнительный полносвёрточный слой, как это приведено в ссылке [1] в заголовке. 

P.S. значения по два, потому что у нас в батче два обучающих примера, длинна каждого из них -- 8...

In [17]:
# часть обучения
import torch.optim as optim

criterion = nn.MSELoss()
# criterion = nn.NLLLoss() # -- этот товарищ требует, чтобы LSTM выдавал классы,
# (числа от 0 до C-1), но как всё-таки его заставить это делать?...
optimizer = optim.SGD(fa.parameters(), lr=0.001, momentum=0.9)

In [19]:
epoch_count = 300
for epoch in range(epoch_count):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.unsqueeze(2)
        labels = labels.unsqueeze(2)
        # print(f"i:{i}, inputs:{inputs}, labels:{labels}")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = fa(inputs)
        # print(f"outputs:{outputs}, squeezed_outputs: {outputs.squeeze()}")
        loss = criterion(outputs.squeeze(), labels.squeeze())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.005
[2,     1] loss: 0.006
[3,     1] loss: 0.006
[4,     1] loss: 0.005
[5,     1] loss: 0.005
[6,     1] loss: 0.005
[7,     1] loss: 0.005
[8,     1] loss: 0.006
[9,     1] loss: 0.006
[10,     1] loss: 0.006
[11,     1] loss: 0.005
[12,     1] loss: 0.006
[13,     1] loss: 0.006
[14,     1] loss: 0.006
[15,     1] loss: 0.005
[16,     1] loss: 0.005
[17,     1] loss: 0.006
[18,     1] loss: 0.006
[19,     1] loss: 0.006
[20,     1] loss: 0.006
[21,     1] loss: 0.005
[22,     1] loss: 0.006
[23,     1] loss: 0.005
[24,     1] loss: 0.006
[25,     1] loss: 0.006
[26,     1] loss: 0.005
[27,     1] loss: 0.005
[28,     1] loss: 0.005
[29,     1] loss: 0.005
[30,     1] loss: 0.005
[31,     1] loss: 0.005
[32,     1] loss: 0.005
[33,     1] loss: 0.005
[34,     1] loss: 0.005
[35,     1] loss: 0.005
[36,     1] loss: 0.005
[37,     1] loss: 0.005
[38,     1] loss: 0.005
[39,     1] loss: 0.005
[40,     1] loss: 0.005
[41,     1] loss: 0.005
[42,     1] loss: 0.005
[

KeyboardInterrupt: 

## Вывод 
для того, чтобы воспользоваться crossentropy или logloss необходимо воспользоваться функцией `F.log_softmax`!!

Это было и в базовом примере:

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

И в примере из документации к NLLLoss:

https://pytorch.org/docs/stable/nn.html#nllloss